In [1]:
%load_ext autoreload 
%autoreload 2

In [2]:
import json
from privacypacking.utils.utils import load_logs, global_metrics
import pandas as pd
from experiments.ray.analysis import load_ray_experiment, load_latest_ray_experiment, load_latest_scheduling_results, load_latest_scheduling_results
import plotly.express as px
from privacypacking.budget.curves import  LaplaceCurve, GaussianCurve, SubsampledGaussianCurve
from privacypacking.budget import Budget, Task, Block
from privacypacking.schedulers.metrics import OverflowRelevance, FlatRelevance
from privacypacking.budget.block_selection import RandomBlocks
from privacypacking.utils.plot import plot_budgets
import yaml
from pathlib import Path
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

In [12]:
df = load_latest_ray_experiment()
df.head()


/home/pierre/privacypacking/logs/ray/run_and_report_2021-11-22_10-15-03


,scheduler,solver,scheduler_n,scheduler_metric,block_selecting_policy,frequency_file,n_allocated_tasks,total_tasks,realized_profit,n_tasks,...,date,timestamp,time_total_s,pid,hostname,node_ip,config,time_since_restore,timesteps_since_restore,iterations_since_restore
0,basic_scheduler,gurobi,100,DominantShares,RandomBlocks,frequencies.yaml,148,300,148.0,300,...,2021-11-22_10-16-19,1637594179,4.268924,2575011,ds-09,128.59.23.59,"{'epsilon': 10, 'delta': 1e-05, 'global_seed':...",4.268924,0,1
1,basic_scheduler,gurobi,100,DominantShares,RandomBlocks,frequencies.yaml,89,100,89.0,100,...,2021-11-22_10-15-51,1637594151,1.565336,2575023,ds-09,128.59.23.59,"{'epsilon': 10, 'delta': 1e-05, 'global_seed':...",1.565336,0,1
2,basic_scheduler,gurobi,100,OverflowRelevance,RandomBlocks,frequencies.yaml,222,300,222.0,300,...,2021-11-22_10-16-48,1637594208,32.409299,2575012,ds-09,128.59.23.59,"{'epsilon': 10, 'delta': 1e-05, 'global_seed':...",32.409299,0,1
3,basic_scheduler,gurobi,100,FlatRelevance,RandomBlocks,frequencies.yaml,367,750,367.0,750,...,2021-11-22_10-17-30,1637594250,28.500733,2576400,ds-09,128.59.23.59,"{'epsilon': 10, 'delta': 1e-05, 'global_seed':...",28.500733,0,1
4,basic_scheduler,gurobi,100,OverflowRelevance,RandomBlocks,frequencies.yaml,154,200,154.0,200,...,2021-11-22_10-16-28,1637594188,11.447010,2575000,ds-09,128.59.23.59,"{'epsilon': 10, 'delta': 1e-05, 'global_seed':...",11.447010,0,1


In [13]:
df["metric"] = df.apply(axis=1, func=lambda row: row["scheduler_metric"] if row["scheduler"] == "basic_scheduler" else "Simplex")

In [14]:
px.line(
    df.sort_values("n_tasks"),
    x="n_tasks",
    y="n_allocated_tasks",
    color="metric",
    log_x=False,
    title="Number of allocated tasks (no profits)<br><sup>10 blocks, offline mixed-curves</sup>"
)